# 1. Data Import and Initial Validation
### The first step in our analysis is to acquire the data and perform a thorough initial check.

- Import the data from its source into a Pandas DataFrame, the primary structure we'll use for analysis.
- Validate the data's basic integrity: We need to ensure there are no immediate red flags, such as widespread missing values or columns with entirely incorrect data types.
- Establish a baseline understanding of the dataset's structure, including its size and the variables it contains.
- Check for duplicated rows in the data

I will load the data from the CSV file marketing_data.csv and then use a combination of Pandas methods to inspect health of the data.



In [109]:
import pandas as pd
import numpy as np
#importing "marketing_data" data to jupyter
df=pd.read_csv('data/raw/marketing_data.csv')
#checking keys of the data
print(df.keys())
#Remving spaces before and after the columns keys in data
df.columns = df.columns.str.strip()
#checking values in each column
duplicate_rows_num = df.duplicated().sum()
print('****************************************************************')
print(f"Number of duplicated rows in the data is : {duplicate_rows_num}")
print('****************************************************************')
print(df.info())


Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', ' Income ',
       'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines',
       'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')
****************************************************************
Number of duplicated rows in the data is : 0
****************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   2240 non-null   int64 
 1   Year_Birth           2240 non-null   int64 
 2   Education     

Based on the result from above cell, spaces in name of varaibles (keys) have been removed. 
Inital inspection showed:
- 5 columns (`Dt_Customer`, `Education`, `Marital_Status`, `country`, `Income`) have `object` type, which might be mixing data types.
- No columns have missing values
- No duplicated rows found in the data
- Values in `Income` column starts with $, which needs to turn to numeric values

# 2. Data type cnoversion
### Since data type are not appropraite for 5 columns as shown above, we need to convert them as belwo. 
- Conver one column (Dt_Customer) to `datetime` type.
- Convert three columns (`Education`, "Marital_Status` and `Country`) into `category` type.
- Remove `$` in the begining of values of `Income` column. then, check the data.
- Conver Income` column into numeric type.
- Inspection entire tabel after data type conversion! 

In [110]:
#convert object columns (Dt_Customer, Education, Marital_Status, Country, Income)
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], format="%m/%d/%y", errors="coerce")
print(df["Dt_Customer"].info())
df["Education"] = df["Education"].astype("category")
df["Marital_Status"] = df["Marital_Status"].astype("category")
df["Country"] = df["Country"].astype("category")
#remove $ from begining of data
df['Income']= df['Income'].str.replace('$', '')
#remove , from data
df['Income']= df['Income'].str.replace(',', '')
#Transfer string type of data in income to numeric type and replace missing values or invalid data to NaN
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')
print(df.dtypes)


<class 'pandas.core.series.Series'>
RangeIndex: 2240 entries, 0 to 2239
Series name: Dt_Customer
Non-Null Count  Dtype         
--------------  -----         
2240 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 17.6 KB
None
ID                              int64
Year_Birth                      int64
Education                    category
Marital_Status               category
Income                        float64
Kidhome                         int64
Teenhome                        int64
Dt_Customer            datetime64[ns]
Recency                         int64
MntWines                        int64
MntFruits                       int64
MntMeatProducts                 int64
MntFishProducts                 int64
MntSweetProducts                int64
MntGoldProds                    int64
NumDealsPurchases               int64
NumWebPurchases                 int64
NumCatalogPurchases             int64
NumStorePurchases               int64
NumWebVisitsMonth               int

According to results of above cell: 
- After data type conversion, `Income` column has 24 missing values, which need to be filled by average income of similar education and marital status. 
- Data in columns (`Marital_Status`, `Education` and `Country`) are good and no missing values
- Data in columns  `Dt_Customer`  are datetime and has no missing values now.


# 3. Handling missing values of columns 
Since `Income` column is the only columns has missing value. I need to handle only this column step by step as below. 
- Calculate avergae income for people from same Marital Status and Education by groupby 
- Find rows with missing values in `Income` column
- Fill missing values in Income column using the calculted values
- Inspect data after one more time

In [111]:
#Finding missing values in all columns of the data
print("Total number of missing value for each column")
print(df.isna().sum())
#finding missing values in income column of the data
missing_values = df['Income'].isnull()
missing_values_Income=df[missing_values][['ID','Income','Marital_Status','Education']]
print('*****************************************************************************')
#print(missing_values_Income)
df_droped = df.dropna(subset=['Income'])
# Calculate average value of income for same Marital_Status and Education
avergae_mat_Edu=df_droped[['Income','Marital_Status','Education']].groupby(['Marital_Status','Education'], observed=True).mean(numeric_only=True)
#filling mising values in income columns by average income of the same education and martial status
missing_equvalent_avg=pd.merge(missing_values_Income,avergae_mat_Edu,on=['Marital_Status','Education'],how='left')
missing_equvalent_avg=missing_equvalent_avg.drop(columns='Income_x')
missing_equvalent_avg=missing_equvalent_avg.rename(columns={'Income_y': 'Income'})
df = df.merge(
    missing_equvalent_avg[['ID', 'Income']], 
    on='ID', 
    how='left', 
    suffixes=('', '_new')
)
# Fill missing values with the new calculated values
df['Income'] = df['Income_new'].combine_first(df['Income'])
df = df.drop('Income_new', axis=1)
print("Total number of missing value after filling for each column")
print(df.isnull().sum())



Total number of missing value for each column
ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Response                0
Complain                0
Country                 0
dtype: int64
*****************************************************************************
Total number of missing value after filling for each column
ID                     0
Year_Birth             0
Education              0


After handling missing values, inspection showed that there is no missing values in any column. Thus, data is clean and ready for analysis/modeling.

# 4. Define new columns and calculate values of the columns
For our analysis, we need to define and calculate four new columns including `Number_children`, `Age`, `total_spending`, and `total_purchases`.
- `Number_children` is defined by adding `Kidhome` and `Teenhome`
- `Age` is defined by subtracting current date from `Year_Birth`
- `total_spending` is defined by adding all expenses
- `total_purchases` is defined by adding all purchases

At the end, we need to check data frame to see the new columns are added by printing 5 first rows of the data. 


In [112]:
from datetime import date
tody_date=date.today()
#Create varaibles for number of children, age, toal spending, and total purchase for customers and added them to df
df['Number_children']=df['Kidhome']+df['Teenhome']
df['Age']=tody_date.year-df['Year_Birth']
df['total_spending']=df['MntWines']+df['MntFruits']+df['MntMeatProducts']+df['MntFishProducts']+df['MntSweetProducts']+df['MntGoldProds']
df['total_purchases']=df['NumWebPurchases']+df['NumCatalogPurchases']+df['NumStorePurchases']
print('*****************print 5 rows of data (number of children,age, toal spending and total purchase for customers)')
print(df.head())


*****************print 5 rows of data (number of children,age, toal spending and total purchase for customers)
      ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0   1826        1970  Graduation       Divorced  84835.0        0         0   
1      1        1961  Graduation         Single  57091.0        0         0   
2  10476        1958  Graduation        Married  67267.0        0         1   
3   1386        1967  Graduation       Together  32474.0        1         1   
4   5371        1989  Graduation         Single  21474.0        1         0   

  Dt_Customer  Recency  MntWines  ...  AcceptedCmp5  AcceptedCmp1  \
0  2014-06-16        0       189  ...             0             0   
1  2014-06-15        0       464  ...             0             0   
2  2014-05-13        0       134  ...             0             0   
3  2014-05-11        0        10  ...             0             0   
4  2014-04-08        0         6  ...             0             0   


Insepction of the data by printing 5 first rows of the data, showed that the four new columns have been defined and added to the data correctly 

# 5. Encode categorical variables using ordinal encoding and one-hot encoding.
- Encode categorical variables inclusing `Education` using ordinal encoding 
- Apply onehot encoding to variable`Country` and `Marital_Status`

In [113]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
#ordinal encoding for variable('Education')
#define levels and apply ordinal encoding to 'Education'
ordinal_encoder = OrdinalEncoder(categories=[['2n Cycle', 'Basic', 'Graduation', 'Master', 'PhD']])
df['Ordinal'] = ordinal_encoder.fit_transform(df[['Education']])
df.drop('Education', axis=1, inplace=True)
df.rename(columns={'Ordinal': 'Education'}, inplace=True)
#onehot encoding for variable('Marital_Status')
# Marital_Status column drops and 8 new columns are created (
onehot_encoder = OneHotEncoder()
onehot_encoded = onehot_encoder.fit_transform(df[['Marital_Status']])
df_onehot = pd.DataFrame(onehot_encoded.toarray(), columns=onehot_encoder.get_feature_names_out(['Marital_Status']))
onehot_encoder2 = OneHotEncoder()
onehot_encoded2 = onehot_encoder2.fit_transform(df[['Country']])
df_onehot2 = pd.DataFrame(onehot_encoded2.toarray(), columns=onehot_encoder2.get_feature_names_out(['Country']))
df.drop('Marital_Status', axis=1, inplace=True)
df.drop('Country', axis=1, inplace=True)
df = pd.concat([df, df_onehot, df_onehot2], axis=1)
print('******************shows first five rows of the datasets after converting the columns (Education, Marital_Status, Country)************')
print(df.head(5))
print(f" Number of rows in the converted dataset: {df.shape[0]}")
print(f" Number of columns in the converted dataset: {df.shape[1]}")

******************shows first five rows of the datasets after converting the columns (Education, Marital_Status, Country)************
      ID  Year_Birth   Income  Kidhome  Teenhome Dt_Customer  Recency  \
0   1826        1970  84835.0        0         0  2014-06-16        0   
1      1        1961  57091.0        0         0  2014-06-15        0   
2  10476        1958  67267.0        0         1  2014-05-13        0   
3   1386        1967  32474.0        1         1  2014-05-11        0   
4   5371        1989  21474.0        1         0  2014-04-08        0   

   MntWines  MntFruits  MntMeatProducts  ...  Marital_Status_Widow  \
0       189        104              379  ...                   0.0   
1       464          5               64  ...                   0.0   
2       134         11               59  ...                   0.0   
3        10          0                1  ...                   0.0   
4         6         16               24  ...                   0.0   

   Mar

- `Education` column converted to ordinal column values from 1 to 5
- `Marital_Status` column converted into Marital_Status_Absurd, Marital_Status_Alone, Marital_Status_Divorced, Marital_Status_Married, Marital_Status_Single, Marital_Status_Together, Marital_Status_Widow, Marital_Status_YOLO
- `Country` column converted into 8 columns including Country_AUS, Country_CA, Country_GER, Country_IND, Country_ME, Country_SA, Country_SP, Country_US

# 6. Save cleaned data into a file.
Next juypter notebook will use this data and do second part of the project. 

In [114]:
df.to_parquet('data/interim/cleaned_data.parquet')